# 1. 예보 데이터 2014년 12월 ~ 2021년 4월

In [1]:
import pandas as pd
from glob import glob

In [2]:
def date_gen(month_data, y, name):
    output=pd.DataFrame(columns=['Forecast time','forecast','value'])
    for i, df in enumerate(month_data):
        month = i+12
        year = y
        if month >= 13:
            month = month - 12
            year += 1
        if month <= 11 and year==2016 and name=='Cloud': # 2016년 1월 하늘형태 데이터 누락 
            month = month + 1
        date = f'{year}-{month}-' + df[' format: day'].str.split(' ').str[-1] + ' ' + (df['hour'].astype(int)//100).astype(str) + ':00'
        
        # 시간단위 UTC => GMT
        date = pd.to_datetime(date) + pd.DateOffset(hours=9)
        data_fcst = pd.DataFrame(columns=['Forecast time','forecast','value'])
        data_fcst['Forecast time'] = date
        data_fcst['forecast'] = df['forecast']
        data_fcst['value'] = df['value']
        output = pd.concat([output, data_fcst])
    return output

def del_month(df ,y, name=' '):    # 월 구분 행 추출
    month_rows = [-1]
    month_rows.extend(df[df['hour'].isna()].index)
    month_rows.append(df.shape[0]+1)
    month_data = []
    for i in range(len(month_rows)-1):
        month_data.append(df.loc[month_rows[i]+1:month_rows[i+1]-1])
    return date_gen(month_data, y, name)

def preprocessing(path_list, y):
    # 데이터 불러오기
    data_year_temperature = pd.read_csv(path_list[0],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_Precipitation = pd.read_csv(path_list[1],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_Precrate = pd.read_csv(path_list[2],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_humidity = pd.read_csv(path_list[3],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_windspeed = pd.read_csv(path_list[4],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_winddirection = pd.read_csv(path_list[5],names=[' format: day', 'hour', 'forecast','value'],header=0)
    data_year_cloud = pd.read_csv(path_list[6],names=[' format: day', 'hour', 'forecast','value'],header=0)
    
    if y==2014: #2015년 5월 21일 발표시간 23시 ~ 2015년 5월 31일 발표시간 23시 데이터 추가 
        tmp1=data_year_temperature.iloc[0:25109,:]
        tmp2=data_year_temperature.iloc[25109::,:]
        tmp3=data_year_temperature.iloc[16411:17911,:]
        tmp4=pd.concat([tmp1,tmp3])
        data_year_temperature=pd.concat([tmp4,tmp2]).reset_index(drop=True)

    data_year_temperature=del_month(data_year_temperature, y)
    data_year_Precipitation=del_month(data_year_Precipitation, y)
    data_year_Precrate=del_month(data_year_Precrate, y)
    data_year_humidity=del_month(data_year_humidity, y)
    data_year_windspeed=del_month(data_year_windspeed, y)
    data_year_winddirection=del_month(data_year_winddirection, y)
    data_year_cloud=del_month(data_year_cloud, y, name='Cloud')
    
    # 데이터 결합
    data_year = pd.DataFrame(columns=['Forecast time'])
    data_year['Forecast time']=data_year_temperature['Forecast time']
    data_year['forecast']=data_year_temperature['forecast']
    df_data = pd.merge(data_year, data_year_Precipitation, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_Precrate, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_temperature, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_humidity, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_windspeed, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_winddirection, on=['Forecast time', 'forecast'], how='outer')
    df_data = pd.merge(df_data, data_year_cloud, on=['Forecast time', 'forecast'], how='outer')
    df_data.columns=['Forecast time', 'forecast', 'Precipitation','PrecRate', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']
    df_data=df_data[['Forecast time', 'forecast', 'Temperature','Precipitation', 'PrecRate', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]
    return df_data

In [3]:
dangjin_data_path = './data/dangjin_fcst'
ulsan_data_path = './data/ulsan_fcst'
csv_list = sorted(glob(dangjin_data_path+'/*/*.csv'))
csv_list2= sorted(glob(ulsan_data_path+'/*/*.csv'))
dangjin_fcst = pd.DataFrame(columns=['Forecast time', 'forecast', 'Temperature','Precipitation', 'PrecRate', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud'])
ulsan_fcst = pd.DataFrame(columns=['Forecast time', 'forecast', 'Temperature','Precipitation', 'PrecRate', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud'])

for i in range(7):
    dangjin_fcst = pd.concat([dangjin_fcst, preprocessing(csv_list[i*7:i*7+7], 2014+i)])
    ulsan_fcst = pd.concat([ulsan_fcst, preprocessing(csv_list2[i*7:i*7+7], 2014+i)])

In [4]:
dangjin_fcst.to_csv('data/dangjin_fcst/new_dangjin_fcst.csv')
ulsan_fcst.to_csv('data/ulsan_fcst/new_ulsan_fcst.csv')

# 2. 발전량 데이터 2015년 11월 ~ 2021년 1월

### 주어진 발전량 데이터를 날짜 형식으로 변환

In [5]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

In [6]:
energy = pd.read_csv('./data/base/energy.csv')
energy['time'] = energy['time'].apply(lambda x:convert_time(x))
energy['time']=pd.to_datetime(energy['time']) + pd.DateOffset(hours=1)
energy[['dangjin','ulsan']]=energy[['dangjin','ulsan']].astype(float)
energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 01:00:00,0.0,0.0,0.0,0.0
1,2018-03-01 02:00:00,0.0,0.0,0.0,0.0
2,2018-03-01 03:00:00,0.0,0.0,0.0,0.0
3,2018-03-01 04:00:00,0.0,0.0,0.0,0.0
4,2018-03-01 05:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0.0,0.0
25628,2021-01-31 21:00:00,0.0,0.0,0.0,0.0
25629,2021-01-31 22:00:00,0.0,0.0,0.0,0.0
25630,2021-01-31 23:00:00,0.0,0.0,0.0,0.0


### 추가한 발전량 데이터 (2015년 1월 ~ 2018년 2월) 전처리

In [7]:
past_energy1 = pd.read_csv('./data/energy/한국동서발전 시간대별 태양광 및 풍력 발전량 현황(2018_2019).csv')
past_energy2 = pd.read_csv('./data/energy/한국동서발전 시간대별 태양광 발전량 현황(2015_2017).csv')
past_energy1['시간']=pd.to_datetime(past_energy1['시간'])
past_energy2['시간']=pd.to_datetime(past_energy2['시간'])
past_energy=pd.concat([past_energy1,past_energy2])
past_energy.sort_values(by=['시간'],ascending=True, inplace=True)

dangjin_energy=past_energy[past_energy['태양광명']=='당진태양광']
dangjin_float_energy=past_energy[past_energy['태양광명']=='당진수상태양광']
dangjin_warehouse_energy=past_energy[past_energy['태양광명']=='당진자재창고태양광']
ulsan_energy=past_energy[past_energy['태양광명']=='울산태양광']

ulsan_energy=ulsan_energy[ulsan_energy['시간'].dt.year <= 2018]
ulsan_energy=ulsan_energy[~((ulsan_energy['시간'].dt.year == 2018) & (ulsan_energy['시간'].dt.month >= 3))]
dangjin_energy=dangjin_energy[dangjin_energy['시간'].dt.year <= 2018]
dangjin_energy=dangjin_energy[~((dangjin_energy['시간'].dt.year == 2018) & (dangjin_energy['시간'].dt.month >= 3))]
dangjin_float_energy=dangjin_float_energy[dangjin_float_energy['시간'].dt.year <= 2018]
dangjin_float_energy=dangjin_float_energy[~((dangjin_float_energy['시간'].dt.year == 2018) & (
    dangjin_float_energy['시간'].dt.month >= 3))]
dangjin_warehouse_energy=dangjin_warehouse_energy[dangjin_warehouse_energy['시간'].dt.year <= 2018]
dangjin_warehouse_energy=dangjin_warehouse_energy[~((dangjin_warehouse_energy['시간'].dt.year == 2018) & (
    dangjin_warehouse_energy['시간'].dt.month >= 3))]

In [8]:
dangjin_float_energy.head(5)

,태양광명,시간,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,용량(MW)
1822,당진수상태양광,2018-01-01,0,0,0,0,0,0,0,0,...,245,63,6,0,0,0,0,0,0,NaN
1823,당진수상태양광,2018-01-02,0,0,0,0,0,0,0,0,...,206,78,4,0,0,0,0,0,0,NaN
1824,당진수상태양광,2018-01-03,0,0,0,0,0,0,0,0,...,302,66,8,0,0,0,0,0,0,NaN
1825,당진수상태양광,2018-01-04,0,0,0,0,0,0,0,0,...,96,36,1,0,0,0,0,0,0,NaN
1826,당진수상태양광,2018-01-05,0,0,0,0,0,0,0,0,...,138,90,5,0,0,0,0,0,0,NaN


In [9]:
dangjin_warehouse_energy.head(5)

,태양광명,시간,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,용량(MW)
2187,당진자재창고태양광,2015-01-01,0,0,0,0,0,0,0,0,...,183,53,0,0,0,0,0,0,0,0.7
2188,당진자재창고태양광,2015-01-02,0,0,0,0,0,0,0,0,...,257,103,3,0,0,0,0,0,0,0.7
2189,당진자재창고태양광,2015-01-03,0,0,0,0,0,0,0,0,...,178,42,1,0,0,0,0,0,0,0.7
2190,당진자재창고태양광,2015-01-04,0,0,0,0,0,0,0,0,...,157,52,1,0,0,0,0,0,0,0.7
2191,당진자재창고태양광,2015-01-05,0,0,0,0,0,0,0,0,...,116,13,0,0,0,0,0,0,0,0.7


In [10]:
dangjin_energy.head(5)

,태양광명,시간,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,용량(MW)
0,당진태양광,2015-01-01,0,0,0,0,0,0,0,0,...,218,44,0,0,0,0,0,0,0,1.0
1,당진태양광,2015-01-02,0,0,0,0,0,0,0,0,...,309,58,0,0,0,0,0,0,0,1.0
2,당진태양광,2015-01-03,0,0,0,0,0,0,0,0,...,220,32,0,0,0,0,0,0,0,1.0
3,당진태양광,2015-01-04,0,0,0,0,0,0,0,0,...,193,41,0,0,0,0,0,0,0,1.0
4,당진태양광,2015-01-05,0,0,0,0,0,0,0,0,...,133,8,0,0,0,0,0,0,0,1.0


In [11]:
ulsan_energy.head(5)

,태양광명,시간,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,용량(MW)
4373,울산태양광,2015-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
4374,울산태양광,2015-02-01,0,0,0,0,0,0,0,4,...,222,90,5,0,0,0,0,0,0,0.5
4375,울산태양광,2015-02-02,0,0,0,0,0,0,0,3,...,187,77,4,0,0,0,0,0,0,0.5
4376,울산태양광,2015-02-03,0,0,0,0,0,0,0,2,...,64,17,3,0,0,0,0,0,0,0.5
4377,울산태양광,2015-02-04,0,0,0,0,0,0,0,0,...,49,44,7,0,0,0,0,0,0,0.5


### 주어진 발전량 데이터 형식에 맞게 변환 후 병합

In [12]:
ulsan_energy=ulsan_energy.reset_index(drop=True)
dangjin_energy=dangjin_energy.reset_index(drop=True)
dangjin_float_energy=dangjin_float_energy.reset_index(drop=True)
dangjin_warehouse_energy=dangjin_warehouse_energy.reset_index(drop=True)

dangjin=pd.DataFrame(columns=['시간','dangjin'])
dangjin_floating=pd.DataFrame(columns=['시간','dangjin_floating'])
dangjin_warehouse=pd.DataFrame(columns=['시간','dangjin_warehouse'])
ulsan=pd.DataFrame(columns=['시간','ulsan'])

dangjin['dangjin']=dangjin['dangjin'].astype(float)
dangjin_floating['dangjin_floating']=dangjin_floating['dangjin_floating'].astype(float)
dangjin_warehouse['dangjin_warehouse']=dangjin_warehouse['dangjin_warehouse'].astype(float)
ulsan['ulsan']=ulsan['ulsan'].astype(float)

for i in range(24):
    dangjin=dangjin.append(dangjin_energy,ignore_index = True)
    dangjin_floating=dangjin_floating.append(dangjin_float_energy,ignore_index = True)
    dangjin_warehouse=dangjin_warehouse.append(dangjin_warehouse_energy,ignore_index = True)
    ulsan=ulsan.append(ulsan_energy,ignore_index = True)

dangjin.sort_values(by=['시간'],ascending=True, inplace=True)
dangjin_floating.sort_values(by=['시간'],ascending=True, inplace=True)
dangjin_warehouse.sort_values(by=['시간'],ascending=True, inplace=True)
ulsan.sort_values(by=['시간'],ascending=True, inplace=True)

dangjin.reset_index(drop=True, inplace=True)
dangjin_floating.reset_index(drop=True, inplace=True)
dangjin_warehouse.reset_index(drop=True, inplace=True)
ulsan.reset_index(drop=True, inplace=True)

for i in range(24):
    dangjin.iloc[i::24,0]=pd.to_datetime('20' + dangjin.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    dangjin_floating.iloc[i::24,0]=pd.to_datetime('20' + dangjin_floating.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    dangjin_warehouse.iloc[i::24,0]=pd.to_datetime('20' + dangjin_warehouse.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)
    ulsan.iloc[i::24,0]=pd.to_datetime('20' + ulsan.iloc[i::24,0].dt.strftime('%y-%m-%d')+ ' ' + str(i) + ':00') + pd.DateOffset(hours=1)

    dangjin.iloc[i::24,1]=dangjin.iloc[i::24,i+3].astype(float)
    dangjin_floating.iloc[i::24,1]=dangjin_floating.iloc[i::24,i+3].astype(float)
    dangjin_warehouse.iloc[i::24,1]=dangjin_warehouse.iloc[i::24,i+3].astype(float)
    ulsan.iloc[i::24,1]=ulsan.iloc[i::24,i+3]

dangjin=dangjin.iloc[:,0:2]
dangjin_floating=dangjin_floating.iloc[:,0:2]
dangjin_warehouse=dangjin_warehouse.iloc[:,0:2]
ulsan=ulsan.iloc[:,0:2]
dangjin.columns = ['time', 'dangjin']
dangjin_floating.columns = ['time', 'dangjin_floating']
dangjin_warehouse.columns = ['time', 'dangjin_warehouse']
ulsan.columns = ['time', 'ulsan']

new_energy= pd.DataFrame(columns=['time'])
new_energy['time'] = pd.date_range(start='2015-01-01 01:00:00', end='2018-03-01 00:00:00', freq='H')
new_energy = pd.merge(new_energy, dangjin_floating, on=['time'], how='outer')
new_energy = pd.merge(new_energy, dangjin_warehouse, on=['time'],how='outer')
new_energy = pd.merge(new_energy, dangjin, on=['time'], how='outer')
new_energy = pd.merge(new_energy, ulsan, on=['time'],how='outer')
new_energy = pd.concat([new_energy, energy])
new_energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2015-01-01 01:00:00,NaN,0.0,0.0,0.0
1,2015-01-01 02:00:00,NaN,0.0,0.0,0.0
2,2015-01-01 03:00:00,NaN,0.0,0.0,0.0
3,2015-01-01 04:00:00,NaN,0.0,0.0,0.0
4,2015-01-01 05:00:00,NaN,0.0,0.0,0.0
...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0.0,0.0
25628,2021-01-31 21:00:00,0.0,0.0,0.0,0.0
25629,2021-01-31 22:00:00,0.0,0.0,0.0,0.0
25630,2021-01-31 23:00:00,0.0,0.0,0.0,0.0


In [13]:
new_energy.to_csv('data/energy/new_energy.csv')

In [14]:
new_energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2015-01-01 01:00:00,NaN,0.0,0.0,0.0
1,2015-01-01 02:00:00,NaN,0.0,0.0,0.0
2,2015-01-01 03:00:00,NaN,0.0,0.0,0.0
3,2015-01-01 04:00:00,NaN,0.0,0.0,0.0
4,2015-01-01 05:00:00,NaN,0.0,0.0,0.0
...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0.0,0.0
25628,2021-01-31 21:00:00,0.0,0.0,0.0,0.0
25629,2021-01-31 22:00:00,0.0,0.0,0.0,0.0
25630,2021-01-31 23:00:00,0.0,0.0,0.0,0.0
